In [0]:
import json
import re
import cni_connectors.adls_gen1_connector as connector
import pyspark.sql.functions as f
import crawler.functions as cf
import requests
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from pyspark.sql.types import StructType, StructField, StringType, TimestampType, LongType, DoubleType

In [0]:
var_adls_uri = connector.adls_gen1_connect(spark, dbutils, scope="adls_gen2", dynamic_overwrite="dynamic")

fornecedor_dev
dev-execution


In [0]:
#Parametros de entrado do ADF
var_file = {'namespace':'API_InGuru',
'file_folder':'inguro_vig_acidente_trab',
'extension':'csv',
'column_delimiter':',',
'encoding':'UTF-8',
'null_value':''}

#Caminho de Dev
var_dls = {
    "folders":{"landing":"/tmp/dev/uld",
    "error":"/tmp/dev/err",
    "staging":"/tmp/dev/stg",
    "log":"/tmp/dev/log",
    "raw":"/tmp/dev/raw",
    "trusted":"/tmp/dev/trs",
    "business":"/tmp/dev/biz",
    "landing":"/tmp/dev/uld"          }}
#Parametros do ADF (Nativo)
var_adf = {
    "adf_factory_name": "cnibigdatafactory",
    "adf_pipeline_name": "org_raw_vig_acidente_trab",
    "adf_pipeline_run_id": "60ee3485-4a56-4ad1-99ae-666666666",
    "adf_trigger_id": "62bee9e9-acbb-49cc-80f2-666666666",
    "adf_trigger_name": "62bee9e9-acbb-49cc-80f2-66666666",
    "adf_trigger_time": "2020-06-08T01:42:41.5507749Z",
    "adf_trigger_type": "PipelineActivity"
}

In [0]:
#Acesso ao json do CMD
lnd = var_dls['folders']['landing']
raw = var_dls['folders']['raw']

In [0]:
var_source = "{lnd}/{namespace}/{file_folder}/".format(lnd=lnd, namespace=var_file['namespace'], file_folder=var_file['file_folder'])
var_source

Out[6]: '/tmp/dev/uld/API_InGuru/inguro_vig_acidente_trab/'

In [0]:
#Origem e destino
source_lnd = "{adl_path}{lnd}/{namespace}/{file_folder}".format(adl_path=var_adls_uri, lnd=lnd, namespace=var_file["namespace"],file_folder=var_file["file_folder"])
source_lnd

Out[7]: '[REDACTED]/tmp/dev/uld/API_InGuru/inguro_vig_acidente_trab'

In [0]:
azure_path = 'abfss://datalake@cnibigdatadlsgen2.dfs.core.windows.net'
outputh_path = '{uri}/uds/uniepro/API_InGuru/inguro_vig_acidente_trab'.format(uri=azure_path)

def API_INGURU(url, per_page, start_date, end_date):
    headers = {
    'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.127 Safari/537.36',
    'accept':'application/json',
    'authorization': '6f64d6a047fad369d35c3806f8f8e0560475075a432585973f91caae35b4ad74'
    } 

    params = {
        'per_page': per_page,
        'start_date': start_date,
        'end_date': end_date,
        'sort': '1'
    }
    response = requests.get(url, headers=headers, params=params)
    return response.json()

In [0]:
def divide_data(data_inicial, data_final):
    formato_data = '%d/%m/%Y %H:%M'

    data_inicial = datetime.strptime(data_inicial, formato_data)
    data_final = datetime.strptime(data_final, formato_data)
    intervalo = data_final - data_inicial
    data_final = data_inicial + intervalo // 2
#     data_final = data_final.replace(hour=23, minute=59, second=59) # define o final como o último segundo do minuto
    data_final = data_final.strftime(formato_data)
    data_inicial = data_inicial.strftime(formato_data)
    return (data_inicial, data_final)

In [0]:
def load_big_df(path):
  schema = StructType([
    StructField("id", LongType(), nullable=True),
    StructField("domain", StringType(), nullable=True),
    StructField("title", StringType(), nullable=True),
    StructField("subtitle", StringType(), nullable=True),
    StructField("author", StringType(), nullable=True),
    StructField("content", StringType(), nullable=True),
    StructField("url", StringType(), nullable=True),
    StructField("source", StringType(), nullable=True),
    StructField("source_country", StringType(), nullable=True),
    StructField("source_state", StringType(), nullable=True),
    StructField("crawled_date", StringType(), nullable=True),
    StructField("published_date", StringType(), nullable=True),
    StructField("dh_insertion_raw", StringType(), nullable=True)
  ])

  df = spark.read.schema(schema) \
             .option("mergeSchema", "true") \
             .option("timestampFormat", "INT96") \
             .parquet(path) \
             .withColumn("dh_insertion_raw", f.col("dh_insertion_raw").cast("timestamp"))\
             .withColumn("crawled_date", f.col("crawled_date").cast("timestamp"))
  
  return df

In [0]:
def get_news(data_inicial, data_final, id):
    url = 'https://app.inguru.me/api/v1/taxonomies/nodes/news/'+str(id)

    array_dfs = []

    getted_news = 0
    
    total_news = API_INGURU(url, 1, data_inicial, data_final)['pagination']['total']
    
    if not total_news:
      dados = {
        'author': np.nan,
        'content': np.nan,
        'crawled_date': np.nan,
        'domain': np.nan,
        'id': np.nan,
        'published_date': np.nan,
        'source': np.nan,
        'source_country': np.nan,
        'source_state': np.nan,
        'subtitle': np.nan,
        'title': np.nan,
        'url': np.nan,
        'dh_insertion_raw': np.nan
      }
      fail_df = pd.DataFrame(dados, index=[0])
      array_dfs.append(fail_df)
      parcial_data_inicial = parcial_data_final
      parcial_data_final = data_final
    else:
      
      print('Total de notícias: ', total_news)

      parcial_data_inicial = data_inicial
      parcial_data_final = data_final

      while getted_news < total_news:

          parcial_news = total_news

          while parcial_news > 10000:
              print('Parcial de notícias: ', parcial_news)
              print('Data inicial: ', parcial_data_inicial)
              print('Data final: ', parcial_data_final)
              parcial_data_inicial, parcial_data_final = divide_data(parcial_data_inicial, parcial_data_final)
              parcial_news = API_INGURU(url, 1, parcial_data_inicial, parcial_data_final)['pagination']['total']

          print("fazendo requisicao")
          print('Data inicial: ', parcial_data_inicial)
          print('Data final: ', parcial_data_final)
          news = API_INGURU(url, parcial_news, parcial_data_inicial, parcial_data_final)['data']

          #CRIA UM DATAFRAME COM AS NOTICIAS
          noticias = pd.DataFrame(news)
          #ADICIONA NUM ARRAY DE DATAFRAME
          array_dfs.append(noticias)

          #corrige as datas

          parcial_data_inicial = parcial_data_final
          parcial_data_final = data_final

          getted_news += parcial_news
          

    return pd.concat(array_dfs)

### Ingestão de anos anteriores

Teve de ser criado pois a API não carrega todos os anos por alguma limitação.

PRECISA SER RODADO APENAS NA PRIMEIRA VEZ

In [0]:
def insere_news(data_inicial, data_final, id, path):
  p = 0
  
  perpat = 0
  
  url = 'https://app.inguru.me/api/v1/taxonomies/nodes/news/'+str(id)

  array_dfs = []

  getted_news = 0
    
  total_news = API_INGURU(url, 1, data_inicial, data_final)['pagination']['total']
    
  if total_news != 0:
      
    print('Total de notícias: ', total_news)

    parcial_data_inicial = data_inicial
    parcial_data_final = data_final

    while getted_news < total_news:
      parte = '_p' + str(p)
      parcial_news = total_news

      while parcial_news > 10000:
        requisicao = API_INGURU(url, 1, parcial_data_inicial, parcial_data_final)
        print('Parcial de notícias antes: ', parcial_news)
        parcial_data_inicial, parcial_data_final = divide_data(parcial_data_inicial, parcial_data_final)
        parcial_news = requisicao['pagination']['total']
        print('Parcial de notícias depois: ', parcial_news)
        
        if parcial_news == 0:
          parcial_data_inicial = parcial_data_final
          parcial_data_final = data_final
          parcial_news = 10001

      print(parcial_news)
      print('Data inicial: ', parcial_data_inicial)
      print('Data final: ', parcial_data_final)
      newsget = API_INGURU(url, parcial_news, parcial_data_inicial, parcial_data_final)
      

      news = newsget['data']

      #CRIA UM DATAFRAME COM AS NOTICIAS
      noticias = pd.DataFrame(news)
      #corrige as datas
      parcial_data_inicial = parcial_data_final
      parcial_data_final = data_final

      getted_news += parcial_news

      dh_insertion_raw = var_adf["adf_trigger_time"].split(".")[0]

      schema = StructType([
        StructField("id", LongType(), nullable=True),
        StructField("domain", StringType(), nullable=True),
        StructField("title", StringType(), nullable=True),
        StructField("subtitle", StringType(), nullable=True),
        StructField("author", StringType(), nullable=True),
        StructField("content", StringType(), nullable=True),
        StructField("url", StringType(), nullable=True),
        StructField("source", StringType(), nullable=True),
        StructField("source_country", StringType(), nullable=True),
        StructField("source_state", StringType(), nullable=True),
        StructField("crawled_date", StringType(), nullable=True),
        StructField("published_date", StringType(), nullable=True)
      ])

      saveDF = spark.createDataFrame(noticias, schema)
      saveDF = saveDF.withColumn("dh_insertion_raw", f.lit(dh_insertion_raw).cast("string"))
      saveDF.write.parquet(path + parte, "append")
      print("Total:", total_news)
      print("Inseridas:", getted_news)
      perpat += parcial_news
      
      if perpat >= 1000000:
        p += 1
        perpat = 0
    
  else:
    print("Sem dados")


In [0]:
graph_ids = []
graph_names = []
graph_dfs = []

nodeID = '930' #Declaração do ID

url_nodeID = 'https://app.inguru.me/api/v1/taxonomies/nodes/'+str(nodeID)
IDs = API_INGURU(url = url_nodeID, per_page=1, start_date='', end_date='')['data'][0]
graph_ids.append(IDs['id'])
graph_names.append(IDs['name'].replace(" ", "_"))

#Passa por cada "filho" no nó principal, caso não tenha nenhum, nada acontece.
for node_children in IDs['children']:
  graph_ids.append(node_children['id'])
  graph_names.append(node_children['name'].replace(" ", "_"))

data_inicial = '26/04/2021 00:00'
data_final = datetime.now().strftime("%d/%m/%Y %H:%M")

for i, id in enumerate(graph_ids):

  path = source_lnd + "_" + str(graph_names[i])
  insere_news(data_inicial, data_final, id, path)


Total de notícias:  20171717
Parcial de notícias antes:  20171717
Parcial de notícias depois:  20171717
Parcial de notícias antes:  20171717
Parcial de notícias depois:  2192031
Parcial de notícias antes:  2192031
Parcial de notícias depois:  340189
Parcial de notícias antes:  340189
Parcial de notícias depois:  224250
Parcial de notícias antes:  224250
Parcial de notícias depois:  89786
Parcial de notícias antes:  89786
Parcial de notícias depois:  58479
Parcial de notícias antes:  58479
Parcial de notícias depois:  45296
Parcial de notícias antes:  45296
Parcial de notícias depois:  37695
Parcial de notícias antes:  37695
Parcial de notícias depois:  27683
Parcial de notícias antes:  27683
Parcial de notícias depois:  11140
Parcial de notícias antes:  11140
Parcial de notícias depois:  9019
9019
Data inicial:  26/04/2021 00:00
Data final:  26/04/2021 08:00
Total: 20171717
Inseridas: 9019
Parcial de notícias antes:  20171717
Parcial de notícias depois:  20167084
Parcial de notícias an

In [0]:
for id in graph_ids:
  print(id)

8785
8787
8789
8786
8788


### Rotina para últimos 7 dias

In [0]:
#SALVANDO EM: /tmp/dev/uld/API_InGuru/inguro_vig_acidente_trab_NOME_DO_NO
# /tmp/dev/uld/API_InGuru/inguro_vig_acidente_trab_NODO_RAIZ
# /tmp/dev/uld/API_InGuru/inguro_vig_acidente_trab_Acidentes_de_Trabalho_Judicializados
# /tmp/dev/uld/API_I

graph_ids = []
graph_names = []
graph_dfs = []

nodeID = '930' #Declaração do ID

url_nodeID = 'https://app.inguru.me/api/v1/taxonomies/nodes/'+str(nodeID)
IDs = API_INGURU(url = url_nodeID, per_page=1, start_date='', end_date='')['data'][0]
graph_ids.append(IDs['id'])
graph_names.append(IDs['name'].replace(" ", "_"))

#Passa por cada "filho" no nó principal, caso não tenha nenhum, nada acontece.
for node_children in IDs['children']:
  graph_ids.append(node_children['id'])
  graph_names.append(node_children['name'].replace(" ", "_"))

#Obtém as notícias de cada nó encontrado começando pelo nó principal.
#Cria um df com as notícias obtidas e salva em um array de dfs.
end_date = datetime.now().strftime("%d/%m/%Y %H:%M")
start_date = (datetime.now() - timedelta(days=7)).strftime("%d/%m/%Y %H:%M")
for id in graph_ids:
  url = 'https://app.inguru.me/api/v1/taxonomies/nodes/news/'+str(id)
  temp_df = get_news(start_date,end_date, id).fillna("Não informado")
  graph_dfs.append(spark.createDataFrame(temp_df))


Total de notícias: 485995
Parcial de notícias: 485995
Data inicial: 16/02/2023 02:26
Data final: 23/02/2023 02:26
Parcial de notícias: 268280
Data inicial: 16/02/2023 02:26
Data final: 19/02/2023 14:26
Parcial de notícias: 187374
Data inicial: 16/02/2023 02:26
Data final: 17/02/2023 20:26
Parcial de notícias: 99080
Data inicial: 16/02/2023 02:26
Data final: 16/02/2023 23:26
Parcial de notícias: 43352
Data inicial: 16/02/2023 02:26
Data final: 16/02/2023 12:56
Parcial de notícias: 10265
Data inicial: 16/02/2023 02:26
Data final: 16/02/2023 07:41
fazendo requisicao
Data inicial: 16/02/2023 02:26
Data final: 16/02/2023 05:03
Parcial de notícias: 485995
Data inicial: 16/02/2023 05:03
Data final: 23/02/2023 02:26
Parcial de notícias: 267665
Data inicial: 16/02/2023 05:03
Data final: 19/02/2023 15:44
Parcial de notícias: 189802
Data inicial: 16/02/2023 05:03
Data final: 17/02/2023 22:23
Parcial de notícias: 100466
Data inicial: 16/02/2023 05:03
Data final: 17/02/2023 01:43
Parcial de notícias: 54755
Data inicial: 16/02/2023 05:03
Data final: 16/02/2023 15:23
Parcial de notícias: 21422
Data inicial: 16/02/2023 05:03
Data final: 16/02/2023 10:13
fazendo requisicao
Data inicial: 16/02/2023 05:03
Data final: 16/02/2023 07:38
Parcial de notícias: 485995
Data inicial: 16/02/2023 07:38
Data final: 23/02/2023 02:26
Parcial de notícias: 264513
Data inicial: 16/02/2023 07:38
Data final: 19/02/2023 17:02
Parcial de notícias: 187733
Data inicial: 16/02/2023 07:38
Data final: 18/02/2023 00:20
Parcial de notícias: 97113
Data inicial: 16/02/2023 07:38
Data final: 17/02/2023 03:59
Parcial de notícias: 63826
Data inicial: 16/02/2023 07:38
Data final: 16/02/2023 17:48
Parcial de notícias: 32132
Data inicial: 16/02/2023 07:38
Data final: 16/02/2023 12:43
Parcial de notícias: 14814
Data inicial: 16/02/2023 07:38
Data final: 16/02/2023 10:10
fazendo requisicao
Data inicial: 16/02/2023 07:38
Data final: 16/02/2023 08:54
Parcial de notícias: 485995
Data inicial: 16/02/2023 08:54
Data final: 23/02/2023 02:26
Parcial de notícias: 259390
Data inicial: 16/02/2023 08:54
Data final: 19/02/2023 17:40
Parcial de notícias: 182955
Data inicial: 16/02/2023 08:54
Data final: 18/02/2023 01:17
Parcial de notícias: 92511
Data inicial: 16/02/2023 08:54
Data final: 17/02/2023 05:05
Parcial de notícias: 64582
Data inicial: 16/02/2023 08:54
Data final: 16/02/2023 18:59
Parcial de notícias: 32477
Data inicial: 16/02/2023 08:54
Data final: 16/02/2023 13:56
Parcial de notícias: 16833
Data inicial: 16/02/2023 08:54
Data final: 16/02/2023 11:25
fazendo requisicao
Data inicial: 16/02/2023 08:54
Data final: 16/02/2023 10:09
Parcial de notícias: 485995
Data inicial: 16/02/2023 10:09
Data final: 23/02/2023 02:26
Parcial de notícias: 252676
Data inicial: 16/02/2023 10:09
Data final: 19/02/2023 18:17
Parcial de notícias: 175927
Data inicial: 16/02/2023 10:09
Data final: 18/02/2023 02:13
Parcial de notícias: 86380
Data inicial: 16/02/2023 10:09
Data final: 17/02/2023 06:11
Parcial de notícias: 62933
Data inicial: 16/02/2023 10:09
Data final: 16/02/2023 20:10
Parcial de notícias: 32297
Data inicial: 16/02/2023 10:09
Data final: 16/02/2023 15:09
Parcial de notícias: 17151
Data inicial: 16/02/2023 10:09
Data final: 16/02/2023 12:39
fazendo requisicao
Data inicial: 16/02/2023 10:09
Data final: 16/02/2023 11:24
Parcial de notícias: 485995
Data inicial: 16/02/2023 11:24
Data final: 23/02/2023 02:26
Parcial de notícias: 245432
Data inicial: 16/02/2023 11:24
Data final: 19/02/2023 18:55
Parcial de notícias: 168197
Data inicial: 16/02/2023 11:24
Data final: 18/02/2023 03:09
Parcial de notícias: 80566
Data inicial: 16/02/2023 11:24
Data final: 17/02/2023 07:16
Parcial de notícias: 59557
Data inicial: 16/02/2023 11:24
Data final: 16/02/2023 21:20
Parcial de notícias: 31783
Data inicial: 16/02/2023 11:24
Data final: 16/02/2023 16:22
Parcial de notícias: 15594
Data inicial: 16/02/2023 11:24
Data final: 16/02/2023 13:53
fazendo requisicao
Data inicial: 16/02/2023 11:24
Data final: 16/02/2023 12:38
P

In [0]:
#Inserção de hora nas tabelas.
dh_insertion_raw = var_adf["adf_trigger_time"].split(".")[0] #Obtendo hora do sistema

for i in range(len(graph_dfs)):
  graph_dfs[i] = graph_dfs[i].withColumn("dh_insertion_raw", f.lit(dh_insertion_raw).cast("string")) #Gravando hora em cada um dos dataframes

In [0]:
path = source_lnd + "_" + str(graph_names[0])
old_df = load_big_df(path)
max_date = old_df.select(f.max("crawled_date")).collect()[0][0]
min_date = max_date - timedelta(days=7)
print(max_date, min_date)
old_df.filter(old_df["crawled_date"] > min_date)
diff_df = graph_dfs[0].join(old_df, on='id', how='left_anti')
diff_df.write.parquet(path, 'append')

In [0]:
#Adição de novos registros

for i in range(len(graph_dfs)):
  path = source_lnd + "_" + str(graph_names[i])
  old_df = load_big_df(path)
#   old_df = spark.read.parquet(path)
  max_date = old_df.select(f.max("crawled_date")).collect()[0][0]
  
  min_date = max_date - timedelta(days=7)
  print(max_date, min_date)
  old_df.filter(old_df["crawled_date"] > min_date)
  
  diff_df = graph_dfs[i].join(old_df, on='id', how='left_anti')
  
  diff_df.write.parquet(path, 'append')

In [0]:
# CARREGAMENTO DAS TABELAS PARA VER SE FORAM SALVAS CORRETAMENTE
test_dfs = []
for i, df in enumerate(graph_dfs):
  path = source_lnd + "_" + str(graph_names[i])
  test_dfs.append(spark.read.parquet(path))
  print(path)

In [0]:
print(test_dfs[0].count())

In [0]:
#ZERAR A BASE

dh_insertion_raw = var_adf["adf_trigger_time"].split(".")[0]

data = {
  'id': 0,
  'domain': 'Não informado',
  'title': 'Não informado',
  'subtitle': 'Não informado',
  'author': 'Não informado',
  'content': 'Não informado',
  'url': 'Não informado',
  'source': 'Não informado',
  'source_country': 'Não informado',
  'source_state': 'Não informado',
  'crawled_date': '1900-01-01 00:00:00',
  'published_date': '1900-01-01 00:00:00',
  'dh_insertion_raw': '2020-06-08T01:42:41.000+0000'
}
schema = StructType([
  StructField("id", LongType(), nullable=True),
  StructField("domain", StringType(), nullable=True),
  StructField("title", StringType(), nullable=True),
  StructField("subtitle", StringType(), nullable=True),
  StructField("author", StringType(), nullable=True),
  StructField("content", StringType(), nullable=True),
  StructField("url", StringType(), nullable=True),
  StructField("source", StringType(), nullable=True),
  StructField("source_country", StringType(), nullable=True),
  StructField("source_state", StringType(), nullable=True),
  StructField("crawled_date", StringType(), nullable=True),
  StructField("published_date", StringType(), nullable=True),
  StructField("dh_insertion_raw", StringType(), nullable=True)
  ])

data['id'] = int(data['id'])

# dfff = pd.DataFrame(data, index=[0])
spark_dfff = spark.createDataFrame([tuple(data.values())], schema=schema)
for i, df in enumerate(graph_dfs):
  path = source_lnd + "_" + str(graph_names[i]) + "_p0"
  spark_dfff.write.parquet(path, mode="overwrite")
  

In [0]:
path = source_lnd + "_" + str(graph_names[2]) + "_p0"
old_df = spark.read.parquet(path)

---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-4243246020355238>:2
      1 path = source_lnd + "_" + str(graph_names[2]) + "_p0"
----> 2 old_df = spark.read.parquet(path)

File /databricks/spark/python/pyspark/instrumentation_utils.py:48, in _wrap_function.<locals>.wrapper(*args, **kwargs)
     46 start = time.perf_counter()
     47 try:
---> 48     res = func(*args, **kwargs)
     49     logger.log_success(
     50         module_name, class_name, function_name, time.perf_counter() - start, signature
     51     )
     52     return res

File /databricks/spark/python/pyspark/sql/readwriter.py:458, in DataFrameReader.parquet(self, *paths, **options)
    447 int96RebaseMode = options.get("int96RebaseMode", None)
    448 self._set_opts(
    449     mergeSchema=mergeSchema,
    450     pathGlobFilter=pathGlobFilter,
   (...)
    455     int96RebaseMode=int96RebaseMode,
  

In [0]:
old_df.count()

Out[25]: 1

In [0]:
spark_dfff.write.parquet(path, mode="overwrite")